In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, BatchNormalization, Activation, UpSampling2D, Concatenate, AveragePooling2D, Input, concatenate, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import backend as K

# Custom Convolution Block
class ConvBlock(Layer):
    def __init__(self, filters, kernel_size, dilation_rate, batch=False, padding='same', activation='relu', use_bias=False, **kwargs):
        super(ConvBlock, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.dilation_rate = dilation_rate
        self.batch = batch
        self.padding = padding
        self.activation = activation
        self.use_bias = use_bias
        
        self.conv = Conv2D(filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding=padding, use_bias=use_bias, kernel_initializer='he_normal')
        self.norm = BatchNormalization()
        self.act = Activation(activation)
    
    def call(self, inputs):
        x = self.conv(inputs)
        if self.batch:
            x = self.norm(x)
            x = self.act(x)
            return x
        else:
            x = self.act(x)
            return x
            
    def get_config(self):
        config = super(ConvBlock, self).get_config()
        config.update({
            "filters": self.filters,
            "kernel_size": self.kernel_size,
            "dilation_rate": self.dilation_rate,
            "batch": self.batch,
            "padding": self.padding,
            "activation": self.activation,
            "use_bias": self.use_bias
        })
        return config

# Custom Dilated Spatial Pyramid Pooling
class DilatedSpatialPyramidPooling(Layer):
    def __init__(self, filters=256, **kwargs):
        super(DilatedSpatialPyramidPooling, self).__init__(**kwargs)
        self.conv1 = ConvBlock(filters, 1, dilation_rate=1, batch=True)
        self.conv2 = ConvBlock(filters, 3, dilation_rate=6, batch=True)
        self.conv3 = ConvBlock(filters, 3, dilation_rate=12, batch=True)
        self.conv4 = ConvBlock(filters, 3, dilation_rate=18, batch=True)
        self.final_conv = ConvBlock(filters, 1, dilation_rate=1, batch=True)

    def call(self, inputs):
        x1 = self.conv1(inputs)
        x2 = self.conv2(inputs)
        x3 = self.conv3(inputs)
        x4 = self.conv4(inputs)
        x = concatenate([x1, x2, x3, x4], axis=-1)
        x = self.final_conv(x)
        return x

    def get_config(self):
        config = super(DilatedSpatialPyramidPooling, self).get_config()
        return config


In [11]:
# Input layer definition
input_shape = (256, 256, 3)  # Update this to your specific input shape
inputs = Input(shape=input_shape)

# Example model building using your custom layers
x = ConvBlock(64, (3, 3), dilation_rate=1)(inputs)
x = DilatedSpatialPyramidPooling()(x)
outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary to verify its structure
print(model.summary())


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_block_10 (ConvBlock)       │ (None, 256, 256, 64)   │         1,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dilated_spatial_pyramid_poolin… │ (None, 256, 256, 256)  │       726,016 │
│ (DilatedSpatialPyramidPooling)  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 256, 256, 1)    │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 728,001 (2.78 MB)

 Trainable params: 725,441 (2.77 MB)

 Non-trainable params: 2,560 (10.00 KB)

None


In [12]:
model.load_weights('model/model_weights.h5')


ValueError: Layer count mismatch when loading weights from file. Model expected 3 layers, found 89 saved layers.